# GLADtoTEXT in Google Colab

Complete setup and usage guide for Google Colab

## Step 1: Install Dependencies and Build

In [ ]:
# Install build dependencies
!apt-get update -qq
!apt-get install -y build-essential g++ git

# Clone repository
!git clone https://github.com/vksssd/GLADtoTEXT.git
%cd GLADtoTEXT

# Build C++ binaries
!make all

# Install Python dependencies
!pip install -q pybind11 numpy

# Install Python package
!pip install -q .

## Step 2: Create Training Data

In [ ]:
%%writefile train.txt
__greeting hello
__greeting hi there
__greeting hey
__greeting good morning
__greeting what's up
__farewell goodbye
__farewell bye
__farewell see you later
__farewell take care
__question how are you
__question what's your name
__question can you help me
__thanks thank you
__thanks thanks a lot
__thanks appreciate it

## Step 3: Train Model (C++ CLI)

In [ ]:
# Train supervised model
!./gladtotext supervised -input train.txt -output chatbot -dim 50 -epoch 20 -lr 0.1

## Step 4: Use Python API for Inference

In [ ]:
import gladtotext

# Load model
model = gladtotext.Model()
model.load_model("chatbot.bin")

# Get model info
info = model.get_info()
print(f"Supervised: {info['supervised']}")
print(f"Labels: {info['labels']}")
print(f"Dimension: {info['dim']}")
print()

## Step 5: Make Predictions

In [ ]:
# Test predictions
test_texts = [
    "hello friend",
    "goodbye everyone",
    "how are you doing",
    "thanks for your help",
    "hey there"
]

for text in test_texts:
    predictions = model.predict(text, k=2)
    print(f"Text: '{text}'")
    for label, score in predictions:
        print(f"  {label}: {score:.4f}")
    print()

## Step 6: Interactive Testing

In [ ]:
# Interactive prediction
def predict_text(text):
    predictions = model.predict(text, k=3)
    print(f"Input: '{text}'")
    print("Predictions:")
    for i, (label, score) in enumerate(predictions, 1):
        print(f"  {i}. {label}: {score:.4f}")

# Try it
predict_text("hi how are you")
predict_text("see you tomorrow")
predict_text("thank you so much")

## Advanced: Train with More Options

In [ ]:
# Train with sentence encoding (better for word order)
!./gladtotext supervised -input train.txt -output chatbot_sentence \
  -dim 50 -epoch 20 -lr 0.1 -sentence -batch 4

# Load and test
model_sentence = gladtotext.Model()
model_sentence.load_model("chatbot_sentence.bin")

# Compare predictions
text = "hello how are you"
print("Standard model:")
print(model.predict(text, k=1))
print("\nSentence encoding model:")
print(model_sentence.predict(text, k=1))

## Download Model (Optional)

In [ ]:
# Download trained model to your computer
from google.colab import files
files.download('chatbot.bin')